In [1]:
import pointCollection as pc
import matplotlib.pyplot as plt
import numpy as np
import re
import glob
import h5py
import scipy.ndimage as snd
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

In [3]:
def smooth_misfit(D, w, field='z', kernel_type='gauss'):
    dx=np.diff(D.x[0:2])
    if kernel_type=='gauss':
        xk=np.arange(-3*w, 3*w+dx, dx)
        K=np.exp(-xk**2/2/w**2)
    else:
        K=np.ones(int(w/dx))
    K /= np.sum(K)
    temp=np.squeeze(getattr(D, field)**2)
    temp[~np.isfinite(temp)]=0
    temp1=np.isfinite(np.squeeze(getattr(D, field))).astype(float)
    sR=snd.convolve( snd.convolve(temp, K[:, np.newaxis], mode='constant'), K[:, np.newaxis].T, mode='constant')
    temp1 = snd.convolve( snd.convolve(temp1, K[:, np.newaxis], mode='constant'), K[:, np.newaxis].T, mode='constant')
    sR[temp1 > 0] /= temp1[temp1>0]
    return np.sqrt(sR)

In [14]:
#maps={}
base='/home/besmith4/nobackup/ATL14_test/'
#for sub in ['base_dz0.3x', 'base', 'base_dz3x']:
for sub in [ 'base', 'base_dtt3x', 'base_dtt10x']:
    run_dir=base+sub
    maps[sub]={}
    maps[sub]['z0']=pc.grid.data().from_h5(run_dir+'/z0.h5', group='z0')
    maps[sub]['dz']=pc.grid.data().from_h5(run_dir+'/dz.h5', group='dz')

In [15]:
maps[sub]['dz']

<class 'pointCollection.grid.data.data'> with shape (321, 321, 6),
with fields:
['count', 'dz', 'misfit_rms', 'misfit_scaled_rms', 'sigma_dz']

In [16]:
plt.figure( figsize=[8, 6]); 
#for count, sub in enumerate(['base_dz0.3x', 'base', 'base_dz3x']):
for count, sub in enumerate([ 'base', 'base_dtt3x', 'base_dtt10x']):
    plt.subplot(3,4,4*count+1)
    plt.imshow(np.gradient(maps[sub]['z0'].z0[:,:,0])[0], extent=maps[sub]['z0'].extent, vmin=-3, vmax=3,  origin='lower',cmap='gray')
    plt.subplot(3,4,4*count+2)
    plt.imshow(maps[sub]['dz'].dz[:,:,4]- maps[sub]['dz'].dz[:,:,0], extent=maps[sub]['dz'].extent, vmin=-3, vmax=3,  origin='lower',cmap='Spectral')
    plt.subplot(3,4,4*count+3)
    plt.imshow(maps[sub]['dz'].sigma_dz[:,:,4], extent=maps[sub]['dz'].extent, clim=[0, 0.05], origin='lower', cmap='magma')    
    plt.subplot(3,4,4*count+4)
    if not hasattr(maps[sub]['z0'],'misfit_rms_s'):
        maps[sub]['z0'].assign({'misfit_rms_s':smooth_misfit(maps[sub]['z0'], 1000, kernel_type='boxcar', field='misfit_scaled_rms')})
    plt.imshow(maps[sub]['z0'].misfit_rms_s, extent=maps[sub]['z0'].extent, vmin=0, vmax=4,  origin='lower')
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
for ii, sub in enumerate(maps.keys()):
    print(f"{sub} {np.nanmedian(maps[sub]['misfit_rms'].zs.ravel())}")



base_dz0.3x 1.743782681865746
base 1.384354851032408
base_dz3x 1.1532535617466944


In [ ]:
plt.figure(); plt.imshow(sR, origin='lower', extent=maps[sub]['misfit_rms'].extent, clim=[0,2]); plt.colorbar()

In [ ]:
E=pc.grid.data().from_h5('/home/besmith4/nobackup/ATL14_test//base/dz_sigma.h5', group='/dz/sigma' )
E_z0=pc.grid.data().from_h5('/home/besmith4/nobackup/ATL14_test//base/z0_sigma.h5', group='/z0/sigma', field_mapping={'z0':'z0'} )

In [ ]:
plt.imshow(E.dz[:,:,1], origin='lower', extent=E.extent, clim=[0, 0.07])
plt.colorbar(label='error for 2019.5')
plt.plot(E.x, np.ones_like(E.x)*E.y[20],'w')
plt.figure(); 
for ii in range(E.dz.shape[2]-1):
    plt.plot(E.x, E.dz[20,:, ii], label=f't={E.t[ii]}')

plt.legend()
plt.gca().set_xlabel('x, m')
plt.gca().set_ylabel('dz error, m')
plt.gca().set_ylim([0, 0.25])


In [ ]:

fig, ax=plt.subplots(4,6, constrained_layout=True, figsize=[18,12])
for ii in range(6):
    hi_dz=ax[0,ii].imshow(E.dz[:,:,ii], origin='lower', extent=E.extent, vmin=0, vmax=0.05); #plt.colorbar(label='$\sigma_{dz}$')
    ax[0,ii].set_title('%2.2f' % E.t[ii])
    if ii+1 < E.t.size:
        hi_dz1=ax[1, ii].imshow(E.dzdt_lag1[:,:,ii], origin='lower', extent=E.extent, vmin=0, vmax=0.2); #plt.colorbar(label='$\sigma_{dz/dt}$ lag 1')
        ax[1, ii].set_title('%2.2f to %2.2f' %( E.t[ii], E.t[ii+1]))
    else:
        ax[1,ii].set_visible(False)
    if ii+4 < E.t.size:
        hi_dz4=ax[2, ii].imshow(E.dzdt_lag4[:,:,ii], origin='lower', extent=E.extent, vmin=0, vmax=0.2); #plt.colorbar(label='$\sigma_{dz/dt}$ lag 4')
        ax[2, ii].set_title('%2.2f to %2.2f' %( E.t[ii], E.t[ii+4]))
    else:
        ax[2,ii].set_visible(False)
    if ii > 0:
        ax[3,ii].set_visible(False)
hi_z0=ax[3,0].imshow(E_z0.z0[:,:,0], origin='lower', extent=E_z0.extent, vmin=0, vmax=1)
ax[3, 0].set_title('$z_0$ (2020)')

#plt.tight_layout()
fig.colorbar(hi_dz, ax=ax[0, :], location='right', shrink=0.6, label='dz')
fig.colorbar(hi_dz1, ax=ax[1,0:-1], location='right', shrink=0.6, label='dz/dt, lag 1')
fig.colorbar(hi_dz4, ax=ax[2,0:2], location='right', shrink=0.6, label='dz/dt, lag 4')
fig.colorbar(hi_z0, ax=[ax[3,0]], location='right', shrink=0.6, label='z0')

fig2=plt.figure()
#plt.tight_layout() 

In [ ]:
#print(maps['base']['dz'].t)
plt.figure(); 
plt.plot(E.t, np.squeeze(E.dz[35, 40, :]), label='dz')
plt.plot(E.t+0.125, np.squeeze(E.dzdt_lag1[35, 40, :]), label='dzdt_lag1')
plt.plot(E.t+0.5, np.squeeze(E.dzdt_lag4[35, 40, :]), label='dzdt_lag4')
z0_error2=np.squeeze(E.dz[35, 40, :])**2
z0_error2[:-1] += z0_error2[1:]
z0_error2[-1] = np.NaN
plt.plot(E.t+0.125, np.sqrt(z0_error2)*4, label='4* rss dz error')
plt.legend()
plt.xlabel('year')
plt.ylabel('error est, m/yr')

In [ ]:
plt.imshow(E_z0.z0[:,:,0], origin='lower', extent=E_z0.extent, vmin=0, vmax=1)


In [ ]:
clim=2*np.array([-1, 1])
n_cols=np.ceil(np.sqrt(dz.shape[2]-2)).astype(int)
n_rows=np.ceil((dz.shape[2]-2)/n_cols).astype(int)+1
fig=plt.figure(2, figsize=[6, 8]); plt.clf()
h_im=[]
#h_ax=[]
fig, h_ax = plt.subplots(n_rows, n_cols, num=2)
for tslice in range(dz.shape[2]-2):
    axn=np.unravel_index(tslice, h_ax.shape)
    ax=h_ax[axn]
    #h_ax.append(fig.add_subplot(n_rows, n_cols, tslice+1))
    h_im.append(ax.imshow(dz.z[:,:,tslice+1]-dz.z[:,:,tslice], extent=dz.extent, origin='lower', cmap='Spectral', clim=clim))
    ax.set_title('%2.2f to %2.2f'% (dz.t[tslice], dz.t[tslice+1]))

axn=np.unravel_index(tslice+1, h_ax.shape)
ax=h_ax[axn]
h_im.append(ax.imshow(dz.z[:,:,tslice+1]-dz.z[:,:,0], extent=dz.extent, origin='lower', cmap='Spectral', clim=clim))
ax.set_title('%2.2f to %2.2f'% (dz.t[0], dz.t[tslice+1]))    

fig.tight_layout()
plt.colorbar(h_im[0], ax=h_ax)
h_ax[-1, -1].set_visible(False)


In [ ]:
(z0.y[-1]-z0.y[0])/np.diff(z0.y[0:2])
z0.shape
thefile=glob.glob(run_dir+'/*/E-160*.h5')
print(thefile)

In [ ]:
thefile=glob.glob(run_dir+'/*/E-160_N-1800.h5')[0]
print(thefile)
temp=pc.grid.data().from_h5(thefile, group='dz', field_mapping={'z':'dz','count':'count'})
plt.figure();
plt.imshow(temp.z[:,:,3]-temp.z[:,:,0], origin='lower', extent=dz.extent); plt.colorbar()
N=pc.grid.data().from_h5(thefile, group='dz', field_mapping={'z':'count'})
plt.figure();
#plt.imshow(np.sum(N.z>0, axis=2))
plt.imshow(N.z[:,:,-2], extent=N.extent, origin='lower')
plt.colorbar()


#plt.imshow(np.nansum(temp.z>0, axis=2), origin='lower', extent=dz.extent); plt.colorbar()

In [ ]:
temp.t

In [ ]:
temp0=pc.grid.data().from_h5(thefile, group='z0', field_mapping={'z':'z0'})
print(temp0)
print(temp0.x[-1]-temp0.x[0])

In [ ]:
ctr_dir='/Volumes/ice2/ben/ATL14_test/IS2//U07/z03xlooser_dt10xlooser_40km/centers'
edge_dir='/Volumes/ice2/ben/ATL14_test/IS2//U07/z03xlooser_dt10xlooser_40km/edges'

c_files=['E-160_N-1800.h5',  'E-120_N-1800.h5']
fig=plt.figure(4)
fig.clf()
Dc={}
Bc={}
for file in c_files:
    Dc[file]=pc.grid.data().from_h5(ctr_dir+'/'+file, group='dz', field_mapping={'z':'dz'})
    with h5py.File(ctr_dir+'/'+file,'r') as h5f:
        Bc[file]={'rgt':np.array(h5f['/bias/rgt']),
           'cycle':np.array(h5f['/bias/cycle']),
           'val':np.array(h5f['/bias/val'])}
De={}
Be={}
for file in ['E-140_N-1800.h5']:
    De[file]=pc.grid.data().from_h5(edge_dir+'/'+file, group='dz', field_mapping={'z':'dz'})    
    with h5py.File(ctr_dir+'/'+file,'r') as h5f:
        Be[file]={'rgt':np.array(h5f['/bias/rgt']),
           'cycle':np.array(h5f['/bias/cycle']),
           'val':np.array(h5f['/bias/val'])}
    
for file in files:
    plt.plot(Dc[file].x, Dc[file].z[int(np.ceil(Dc[file].shape[0]/2)),:,2])

for file in ['E-140_N-1800.h5']:
    plt.plot(De[file].x, De[file].z[int(np.ceil(De[file].shape[0]/2)),:,2], '--')




In [ ]:
fig=plt.figure(5)
fig.clf()
Dc0={}
for file in files:
    Dc0[file]=pc.grid.data().from_h5(ctr_dir+'/'+file, group='z0', field_mapping={'z':'z0'})
    
De0={}
for file in ['E-140_N-1800.h5']:
    De0[file]=pc.grid.data().from_h5(edge_dir+'/'+file, group='z0', field_mapping={'z':'z0'})    
  
for file in files:
    plt.plot(Dc0[file].x, Dc0[file].z[int(np.ceil(Dc0[file].shape[0]/2)),:])

for file in ['E-140_N-1800.h5']:
    plt.plot(De0[file].x, De0[file].z[int(np.ceil(De0[file].shape[0]/2)),:], '--')


In [ ]:
len(rgts)

In [ ]:
rc, ic, ie=np.intersect1d(Bc[files[2]]['rgt']+1j*Bc[files[2]]['cycle'], \
                          Be[files[1]]['rgt']+1j*Be[files[1]]['cycle'], return_indices=True)
fig=plt.figure(6); plt.clf()
plt.hist(Bc[files[2]]['val'][ic]-Be[files[1]]['val'][ie])


In [5]:
! ls -lt ~/temp/centers

total 730752
-rw-rw-r-- 1 besmith4 j1008 707264965 Aug 25 17:35 E0_N240.h5
-rw-rw-r-- 1 besmith4 j1008  40815240 Aug 18 23:50 E-720_N-480.h5


In [6]:
! ls -lt ~/temp/decimated/centers

total 134304
-rw-rw-r-- 1 besmith4 j1008 137309295 Aug 25 18:18 E0_N240.h5


In [39]:
dzf=pc.grid.data().from_h5('/home/besmith4/temp/centers/E0_N240.h5', group='dz')
dzd=pc.grid.data().from_h5('/home/besmith4/temp/decimated/centers/E0_N240.h5', group='dz')
z0f=pc.grid.data().from_h5('/home/besmith4/temp/centers/E0_N240.h5', group='z0')
z0d=pc.grid.data().from_h5('/home/besmith4/temp/decimated/centers/E0_N240.h5', group='z0')


In [61]:
for this_dz in [dzf, dzd]:
    fig=plt.figure();
    for i in np.arange(4)+1:
        fig.add_subplot(2,2, i)
        plt.imshow(this_dz.dz[:,:,i]-this_dz.dz[:,:,i-1], clim=[-0.1, 0.1], cmap='Spectral', origin='lower'); 
        plt.title(f'{this_dz.t[i-1]} to {this_dz.t[i]}')
plt.tight_layout()
#plt.figure();
#plt.imshow(dzd.dz[:,:,i]-dzf.dz[:,:,i-1], clim=[-0.1, 0.1], cmap='Spectral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
plt.figure();
plt.imshow(np.gradient(z0d.z0)[1], cmap='gray', origin='lower')
#plt.figure();
#plt.imshow(np.gradient(z0d.z0)[0], cmap='gray')
         

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
plt.figure();
plt.imshow(z0d.z0-z0f.z0, clim=[-.2, 0.2]); plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plt.figure(); plt.plot(z0d.z0[200,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …